In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import VotingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor

In [2]:
df = pd.read_csv('/kaggle/input/human-resources-expe/Human_Resources.csv')
len(df.columns)

35

In [3]:
df2 = df[['Age', 'Attrition', 'Education', 'EducationField', 'JobSatisfaction', 'RelationshipSatisfaction', 'WorkLifeBalance','PerformanceRating']]
df2

,Age,Attrition,Education,EducationField,JobSatisfaction,RelationshipSatisfaction,WorkLifeBalance,PerformanceRating
0,41,Yes,2,Life Sciences,4,1,1,3
1,49,No,1,Life Sciences,2,4,3,4
2,37,Yes,2,Other,3,2,3,3
3,33,No,4,Life Sciences,3,3,3,3
4,27,No,1,Medical,2,4,3,3
...,...,...,...,...,...,...,...,...
1465,36,No,2,Medical,4,3,3,3
1466,39,No,1,Medical,1,1,3,3
1467,27,No,3,Life Sciences,2,2,3,4
1468,49,No,3,Medical,2,4,2,3


In [4]:
df2['EducationField'].unique()

array(['Life Sciences', 'Other', 'Medical', 'Marketing',
       'Technical Degree', 'Human Resources'], dtype=object)

In [5]:
# Reemplazar los valores en la columna "Género"
df2['EducationField'] = df['EducationField'].replace({'Other': 0, 'Life Sciences':1, 'Medical':2, 'Marketing':3, 'Technical Degree':4, 
                                                      'Human Resources':5, 'Human Resources':6 })

    

/tmp/ipykernel_32/52479154.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['EducationField'] = df['EducationField'].replace({'Other': 0, 'Life Sciences':1, 'Medical':2, 'Marketing':3, 'Technical Degree':4,


In [6]:
df2

,Age,Attrition,Education,EducationField,JobSatisfaction,RelationshipSatisfaction,WorkLifeBalance,PerformanceRating
0,41,Yes,2,1,4,1,1,3
1,49,No,1,1,2,4,3,4
2,37,Yes,2,0,3,2,3,3
3,33,No,4,1,3,3,3,3
4,27,No,1,2,2,4,3,3
...,...,...,...,...,...,...,...,...
1465,36,No,2,2,4,3,3,3
1466,39,No,1,2,1,1,3,3
1467,27,No,3,1,2,2,3,4
1468,49,No,3,2,2,4,2,3


In [7]:
df2['Attrition']=df2['Attrition'].apply(lambda x: 1 if x=='Yes' else 0)

/tmp/ipykernel_32/3647747464.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Attrition']=df2['Attrition'].apply(lambda x: 1 if x=='Yes' else 0)


In [8]:
column_names = df2.columns
column_names

Index(['Age', 'Attrition', 'Education', 'EducationField', 'JobSatisfaction',
       'RelationshipSatisfaction', 'WorkLifeBalance', 'PerformanceRating'],
      dtype='object')

In [9]:
df2.head()

,Age,Attrition,Education,EducationField,JobSatisfaction,RelationshipSatisfaction,WorkLifeBalance,PerformanceRating
0,41,1,2,1,4,1,1,3
1,49,0,1,1,2,4,3,4
2,37,1,2,0,3,2,3,3
3,33,0,4,1,3,3,3,3
4,27,0,1,2,2,4,3,3


In [10]:
X = df2.drop('Attrition', axis=1)

In [11]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X.columns = X.columns.astype(str) # En las versiones actuales es necesario para que todas las columnas sean tipo String
X = scaler.fit_transform(X)

In [12]:
X

array([[0.54761905, 0.25      , 0.16666667, ..., 0.        , 0.        ,
        0.        ],
       [0.73809524, 0.        , 0.16666667, ..., 1.        , 0.66666667,
        1.        ],
       [0.45238095, 0.25      , 0.        , ..., 0.33333333, 0.66666667,
        0.        ],
       ...,
       [0.21428571, 0.5       , 0.16666667, ..., 0.33333333, 0.66666667,
        1.        ],
       [0.73809524, 0.5       , 0.33333333, ..., 1.        , 0.33333333,
        0.        ],
       [0.38095238, 0.5       , 0.33333333, ..., 0.        , 1.        ,
        0.        ]])

In [13]:
y=df2['Attrition']
y

0       1
1       0
2       1
3       0
4       0
       ..
1465    0
1466    0
1467    0
1468    0
1469    0
Name: Attrition, Length: 1470, dtype: int64

In [14]:
models = [
    ('decision_tree', DecisionTreeRegressor()),
    ('linear_regression', LinearRegression()),
    ('k_neighbors', KNeighborsRegressor(n_neighbors=5))
]

In [15]:
# Crea un modelo de ensamble con los modelos anteriores
model = VotingRegressor(models)

In [16]:
# Entrena el modelo con los datos
model.fit(X, y)

VotingRegressor(estimators=[('decision_tree', DecisionTreeRegressor()),
                            ('linear_regression', LinearRegression()),
                            ('k_neighbors', KNeighborsRegressor())])

In [17]:
column_names

Index(['Age', 'Attrition', 'Education', 'EducationField', 'JobSatisfaction',
       'RelationshipSatisfaction', 'WorkLifeBalance', 'PerformanceRating'],
      dtype='object')

In [25]:
# Hace una predicción con el modelo
Age = 24
Education = 3
EducationField = 4
JobSatisfaction = 1
RelationshipSatisfaction = 2
WorkLifeBalance = 3
PerformanceRating = 3

In [26]:
predict = model.predict([[Age, Education, EducationField, JobSatisfaction, RelationshipSatisfaction, WorkLifeBalance, PerformanceRating]])
predict

array([-1.92147477])

In [27]:
if abs(predict)<=2:
    print('Se va')
else:
    print('Se queda')

Se va


In [24]:
import joblib

# Guardar la lista de modelos en un archivo
joblib.dump(models, 'modelos.pkl')


['modelos.pkl']